In [1]:
import os
import json
import re
from tqdm import tqdm
import pickle
import itertools
import networkx as nx
from matplotlib import pyplot as plt
import networkx
import pandas as pd
import pickle

In [38]:
#df_texts = pd.read_csv('./dtf/df_texts.csv')
df_info = pd.read_csv('./vc/df_info.csv')

In [64]:
df_sentiment = pd.read_csv('./vc/vc_ru_average_sentiment.csv')
df_sentiment['post_id'] = df_sentiment['post_id'].astype(int)

In [7]:
with open('./vc/vc_extracted_words.pkl', 'rb') as f:
    extracted_words = pickle.load(f)

In [15]:
with open('companies_vc.txt', 'r') as f:
    companies_vc = f.read()

In [16]:
len(companies.split('\n'))

476

In [17]:
companies_vc = companies_vc.split('\n')
companies = set(companies_vc)

In [18]:
stop_words = set(['set', 
                  'далее', 
                  'развитие', 
                  'алгоритмы', 
                  'мария', 
                  'red', 
                  'main', 
                  'стартап', 
                  'бюро', 
                  'id', 
                  'поиск', 
                  'ясно',
                  'интерес',
                  'we',
                  'fl',
                  'too',
                  'успех',
                  'система'])

In [19]:
companies.add('mail.ru')
companies.add('mail')
companies.add('mail.ru group')
companies.add('мэйл')
#companies.add('facebook')
#companies.add('meta')
companies = companies - stop_words

In [20]:
extracted_companies = {}
for key in extracted_words.keys():
    for word in extracted_words[key]:
        if word in companies:
            if not key in extracted_companies.keys():
                extracted_companies[key] = []
            extracted_companies[key].append(word)

In [22]:
for key in extracted_companies.keys():
    extracted_companies[key] = list(set(extracted_companies[key]))

In [23]:
extracted_companies_ = {}
for key in extracted_companies.keys():
    extracted_companies_[key] = ';'.join(extracted_companies[key])

In [50]:
df_companies = pd.DataFrame(extracted_companies_, index=[0]).T.reset_index().rename({0:'companies', 'index': 'doc_id'}, axis=1)

In [51]:
df_companies.to_csv('./vc/df_comp.csv', index=False)

In [52]:
df_companies['companies'] = df_companies['companies'].str.split(';')
df_companies = df_companies.explode('companies').rename({'index': 'doc_id'},axis=1)#.reset_index()

In [53]:
df_info['doc_id'] = df_info['doc_id'].astype(int)
df_companies['doc_id'] = df_companies['doc_id'].astype(int)



In [54]:
df_companies_info = df_companies.merge(df_info, how='left', on='doc_id').groupby('companies').agg({'commentsCount':'mean', 'hitsCount': 'mean', 'likes': 'mean'}).reset_index()

In [56]:
df_companies_info.to_csv('./vc/vc_likes.csv', index=False)

In [65]:
df_sentiment

,post_id,neutral,negative,positive,skip,speech
0,1,0.000000,0.808077,0.000000,0.461027,0.000000
1,158,0.995678,0.000000,0.051855,0.000000,0.000000
2,393,0.000000,0.377551,0.000000,0.468801,0.000000
3,647,0.970041,0.000000,0.000000,0.046136,0.000000
4,757,0.904661,0.000000,0.000000,0.046044,0.000000
...,...,...,...,...,...,...
105838,299968,0.641929,0.233717,0.011210,0.039318,0.000000
105839,299970,0.631669,0.045514,0.100534,0.007674,0.016815
105840,299974,0.698315,0.084865,0.022162,0.020689,0.100033
105841,299975,0.736408,0.167973,0.019059,0.018632,0.003353


In [68]:
df_companies_sent = df_companies.merge(df_sentiment, how='left', left_on='doc_id', right_on='post_id').fillna(0)

In [69]:
df_companies_sent

,doc_id,companies,post_id,neutral,negative,positive,skip,speech
0,159,вконтакте,0.0,0.0,0.0,0.0,0.0,0.0
1,308,макс,0.0,0.0,0.0,0.0,0.0,0.0
2,318,макс,0.0,0.0,0.0,0.0,0.0,0.0
3,322,макс,0.0,0.0,0.0,0.0,0.0,0.0
4,368,одноклассники,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
84792,299982,mail,0.0,0.0,0.0,0.0,0.0,0.0
84793,299986,skyeng,0.0,0.0,0.0,0.0,0.0,0.0
84794,299987,яндекс,0.0,0.0,0.0,0.0,0.0,0.0
84795,299995,spotify,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
comp_sent = df_companies_sent.groupby('companies').agg({'neutral':'mean', 'negative': 'mean', 'positive': 'mean'}).reset_index()

In [74]:
comp_sent.to_csv('./vc/vc_sent.csv', index=False)